In [1]:
import numpy as np
import pandas as pd

from db_handler import DBHandler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans

In [2]:
db = DBHandler()
db.setup_test_table()

R = {}
row = 0
col = 0
count = 0

for x in db.read_data("comp3208-train-small.csv"):
    list_parts = x.strip().split(',')

    if list_parts[2] == "rating":
        continue
    
    user = int(list_parts[0]) - 1
    item = int(list_parts[1]) - 1
    rating = float(list_parts[2])
        
    if int(user) > row:
        row = user
    if item > col:
        col = item
    
    if user in R:
        R[user][item] = rating
        count += 1
    else:
        R[user] = {item: rating}
        count += 1
        
row += 1
col += 1

INFO 2021-05-10 17:53:17,950 logging started
INFO 2021-05-10 17:53:17,960 Setting up test table
INFO 2021-05-10 17:53:17,965 Reading data from database: comp3208-train-small.csv


In [33]:
db = DBHandler()
db.setup_test_table()

row = [] # user
col = [] # item
rating = []

for x in db.read_data("comp3208-train-small.csv"):
    line_split = x.strip().split(',')
    if line_split[2] == "rating":
        continue
    row.append(int(line_split[0]) - 1)
    col.append(int(line_split[1]) - 1)
    rating.append(float(line_split[2]))
    count += 1
    

INFO 2021-02-26 19:12:07,503 logging started
INFO 2021-02-26 19:12:07,505 Setting up test table
INFO 2021-02-26 19:12:08,015 Reading data from database: comp3208-train-small.csv


9425745


In [28]:
from scipy.sparse import coo_matrix

R = coo_matrix((rating, (row, col)), shape=((max(row) + 1), (max(col) + 1)))

In [3]:
user_list = np.empty(shape=count)
movie_list = np.empty(shape=count)
true = np.empty(shape=count)
counter = 0

for user, info in R.items():
    for movie, rating in info.items():
        user_list[counter] = user
        movie_list[counter] = movie
        true[counter] = rating
        counter += 1

In [4]:
# predict random

np.random.seed(0)
pred = np.random.uniform(0.5, 5, count)

In [13]:
# predict 3.5

pred = np.full(shape=count, fill_value=3.5)

In [32]:
ratings_dict = {
    "user": user_list.tolist(),
    "movie": movie_list.tolist(),
    "rating": true.tolist()
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(df[["user", "movie", "rating"]], reader)

sim_options = {
    "name": "cosine",
    "user_based": False,
}

algo = KNNWithMeans(sim_options=sim_options)

trainset = data.build_full_trainset()

algo.fit(trainset)

pred = np.empty(shape=count)

for x in range(0, count):
    pred[x] = algo.predict(user_list[x], movie_list[x]).est

Computing the cosine similarity matrix...
Done computing similarity matrix.


ValueError: setting an array element with a sequence.

In [5]:
print("MAE:  " + str(mean_absolute_error(true, pred)))
print("RMSE: " + str(mean_squared_error(true, pred, squared=False)))
print("MSE:  " + str(mean_squared_error(true, pred)))

MAE:  1.5046945317582245
RMSE: 1.8429486510110766
MSE:  3.3964597302635475
